## Intel-Image-Classification

Importing the required libraries

In [1]:
import numpy as np
import os
import cv2
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

Importing the dataset from local_disk

In [26]:
TRAINDIR = "C:/Users/USER/Documents/intel-image-classification/seg_train"
TESTDIR = "C:/Users/USER/Documents/intel-image-classification/seg_test"
PREDDIR = "C:/Users/USER/Documents/intel-image-classification/seg_pred"
CATEGORIES=['buildings','forest','glacier','mountain','sea','street']

IMG_SIZE = 150
train_data=[]
test_data=[]

def create_data(): # function to create the dataset 
    for category in CATEGORIES:
        train_path=os.path.join(TRAINDIR,category)
        test_path=os.path.join(TESTDIR,category)
        #trian_path=os.path.join(PREDDIR,category)
        class_num = CATEGORIES.index(category) #mapping the dataset to its index
        for img in os.listdir(train_path):
                try:
                    train_img_array = cv2.imread(os.path.join(train_path,img)) #Reading the dataset from path
                    new_train_array = cv2.resize(train_img_array,(IMG_SIZE,IMG_SIZE)) #Reasizing the dataset
                    train_data.append([new_train_array,class_num])
                except Exception as e:
                    pass
        for img in os.listdir(test_path):
                try:
                    test_img_array = cv2.imread(os.path.join(test_path,img))#Reading the dataset from path
                    new_test_array = cv2.resize(test_img_array,(IMG_SIZE,IMG_SIZE))#Reasizing the dataset
                    test_data.append([new_test_array,class_num])
                except Exception as e:
                    pass

Resizing the dataset

In [33]:
create_data()
print(len(train_data))

14034


In [25]:
print(len(test_data))

3000


shuffling the data
preventing the model from learning the dataset as it is arranged

In [27]:
import random
random.shuffle(train_data)
random.shuffle(test_data)

In [39]:
for sample in train_data[:10]:
    print(sample[0])

[[[245 247 247]
  [245 247 247]
  [246 248 248]
  ...
  [248 248 248]
  [245 247 247]
  [245 247 247]]

 [[244 246 246]
  [245 247 247]
  [245 247 247]
  ...
  [248 248 248]
  [245 247 247]
  [245 247 247]]

 [[244 246 246]
  [244 246 246]
  [245 247 247]
  ...
  [248 248 248]
  [245 247 247]
  [245 247 247]]

 ...

 [[ 44  46  47]
  [ 59  61  62]
  [ 46  48  49]
  ...
  [ 43  46  51]
  [ 37  40  45]
  [ 86  89  94]]

 [[ 74  76  77]
  [ 44  46  47]
  [ 40  42  43]
  ...
  [ 14  17  22]
  [ 47  50  55]
  [ 31  34  39]]

 [[ 56  58  59]
  [ 52  54  55]
  [ 57  59  60]
  ...
  [ 23  26  31]
  [ 43  46  51]
  [ 55  58  63]]]
[[[186 121  70]
  [185 122  71]
  [184 121  71]
  ...
  [233 223 223]
  [231 222 219]
  [228 219 216]]

 [[186 123  73]
  [184 123  73]
  [183 122  72]
  ...
  [241 233 233]
  [238 231 228]
  [236 229 226]]

 [[182 123  77]
  [181 122  76]
  [180 121  75]
  ...
  [245 240 239]
  [243 238 235]
  [240 235 232]]

 ...

 [[ 20  36  35]
  [ 32  50  49]
  [ 21  42  39]
  ..

In [40]:
X=[]
y=[]

for features, label in train_data:
    X.append(features)
    y.append(label)
X = np.array(X).reshape(-1,IMG_SIZE,IMG_SIZE,3)
y = np.array(y)

### saving your preprocessed dataset

In [41]:
import pickle
# saving X features
pickle_out = open('X.pickle','wb')
pickle.dump(X,pickle_out)
pickle_out.close()
#saving y label
pickle_out = open('y.pickle','wb')
pickle.dump(y,pickle_out)
pickle_out.close()

In [2]:
# some libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

importing the saved dataset

In [2]:
# importing our preprocessed data
import pickle
X = pickle.load(open('X.pickle','rb'))
y = pickle.load(open('y.pickle','rb'))

In [3]:
#normalizing the X_features
X = X/255.0

### Building the model

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')

])

Compileing the model

In [8]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 72, 64)        36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 34, 34, 32)        18464     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 17, 17, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9248)             

### Training the model

In [13]:
%load_ext tensorboard
import time
from datetime import datetime
from packaging import version

# Define the Keras TensorBoard callback.
#logdir="C:/Users/USER/Documents/intel-image-classification/logs/fit" + datetime.now().strftime("%Y%m%d-%H%M%S")
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir) ,callbacks=[tensorboard_callback]
logdir = os.path.join("logs",datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

model.fit(x=X,y=y,epochs=3,validation_split=0.1,callbacks=[tensorboard_callback])


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Train on 12630 samples, validate on 1404 samples
Epoch 1/3
12630/12630 [==============================] - 1406s 111ms/sample - loss: 0.9957 - accuracy: 0.6107 - val_loss: 1.0043 - val_accuracy: 0.6439
Epoch 2/3
12630/12630 [==============================] - 1475s 117ms/sample - loss: 0.6759 - accuracy: 0.7500 - val_loss: 0.9386 - val_accuracy: 0.6189
Epoch 3/3
12630/12630 [==============================] - 1462s 116ms/sample - loss: 0.5185 - accuracy: 0.8135 - val_loss: 0.7816 - val_accuracy: 0.6923


In [4]:
%load_ext tensorboard
import time
from datetime import datetime
from packaging import version

logdir = os.path.join("logs",datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [23]:
model.save('intel_70.model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: intel_70.model\assets


In [7]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 416), started 2 days, 6:59:35 ago. (Use '!kill 416' to kill it.)

In [5]:
CATEGORIES=['buildings','forest','glacier','mountain','sea','street']
def prepare(filepath):
    IMG_SIZE = 150
    img_array = cv2.imread(filepath)
    new_array = cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
    return new_array.reshape(-1,IMG_SIZE,IMG_SIZE,3)

model = tf.keras.models.load_model('intel_70.model')

prediction = model.predict([prepare('build.jpg')])
print(prediction)

ValueError: Python inputs incompatible with input_signature:
  inputs: (
    Tensor("IteratorGetNext:0", shape=(None, 150, 150, 3), dtype=uint8))
  input_signature: (
    TensorSpec(shape=(None, None, None, 3), dtype=tf.float32, name=None))

In [6]:
CATEGORIES=['buildings','forest','glacier','mountain','sea','street']
def prepare(filepath):
    IMG_SIZE = 150
    img_array = cv2.imread(filepath)
    new_array = cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
    return new_array.reshape(-1,IMG_SIZE,IMG_SIZE,3)

model = tf.keras.models.load_model('intel_70.model')

prediction = model.predict([cv2.imread('build.jpg')])
print(prediction)

ValueError: Error when checking input: expected conv2d_3_input to have 4 dimensions, but got array with shape (150, 150, 3)

In [ ]:
()